<a href="https://colab.research.google.com/github/fedrikkhou/fedrikkhou/blob/main/Indonesian_Stock_IPO_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A) Import Library and Data

In [201]:
import pandas as pd                 # Library used for working with data sets and perform data analysis.
import numpy as np                  # To perform mathematical operasion and statistics
from datetime import datetime       # Used to perform dates and time manipulation

In [202]:
#IDNFinansialsIPOList.csv
#https://drive.google.com/file/d/1uqr851lQrv9R_nrsHdv4TWR_EOhUqsAS/view
url_IDNFinansialsIPOList = 'https://drive.google.com/uc?id=1uqr851lQrv9R_nrsHdv4TWR_EOhUqsAS'
df_IDNFinansialsIPOList = pd.read_csv(url_IDNFinansialsIPOList, sep=';')

#SahamIdxIPOList.csv
#https://drive.google.com/file/d/1U_MJf5qnM7bSLpiFaEdZvsNLFmagujAX/view
url_SahamIdxIPOList = 'https://drive.google.com/uc?id=1U_MJf5qnM7bSLpiFaEdZvsNLFmagujAX'
df_SahamIdxIPOList = pd.read_csv(url_SahamIdxIPOList, sep=';')

#LembarSahamLastPrice.csv
#https://drive.google.com/file/d/1qtgaQUyuh6P9M9pI4p7GGY5e96cWp9VX/view
url_LembarSahamLastPrice = 'https://drive.google.com/uc?id=1qtgaQUyuh6P9M9pI4p7GGY5e96cWp9VX'
df_LembarSahamLastPrice = pd.read_csv(url_LembarSahamLastPrice, sep=';')

#StockSector_Final.xlsx
#https://docs.google.com/spreadsheets/d/e/2PACX-1vSwxoc0ipfDUqQlhOUXd8PmEMSqrupgAltKMcVklHtgmPtELJWyJnzZkWJC2gjfBg/pub?output=xlsx
url_StockSector = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSwxoc0ipfDUqQlhOUXd8PmEMSqrupgAltKMcVklHtgmPtELJWyJnzZkWJC2gjfBg/pub?output=xlsx'
df_StockSector = pd.read_excel(url_StockSector, sheet_name='Sheet1')

#B) Fix Data

##Merge Data

In [203]:
df_SahamIdxIPOList2 = df_SahamIdxIPOList.copy()
df_SahamIdxIPOList2.drop(['No', 'Company Name', 'Sector',
                 'IPO Date', 'Board', 'Underwriters'], axis=1, inplace=True)

df_stocks = pd.merge(df_IDNFinansialsIPOList, df_SahamIdxIPOList2, on='Stock Code')

In [204]:
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 889
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   No                                890 non-null    int64  
 1   Stock Code                        890 non-null    object 
 2   Company Name                      890 non-null    object 
 3   Sector                            890 non-null    object 
 4   Industry_x                        890 non-null    object 
 5   Last Price                        890 non-null    int64  
 6   IPO Date                          890 non-null    object 
 7   IPO Price_x                       890 non-null    int64  
 8   IPO New Shares Issued             890 non-null    int64  
 9   Owner Shares Amount               890 non-null    int64  
 10  Total Shares Amount               890 non-null    int64  
 11  IPO New Shares Issued Percentage  890 non-null    float64
 12  IPO Rais

##1) Fix Data for Stock code BUKK and INCF

In [205]:
#data from IDNFinancial
df_stocks[(df_stocks['Total Number of Shares'] == 'ERROR') | (df_stocks['IPO Number of Shares'] == 'ERROR')]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
383,384,INCF,PT. Indo Komoditi Korpora Tbk,Barang Baku,Perhutanan dan Kertas,50,12/18/1989,8000,1200000,0,...,Basic Industry & Chemicals,Others Basic Industry & Chemicals,-,-,B. Basic Materials,B1. Basic Materials,ERROR,ERROR,ERROR,ERROR
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,ERROR,ERROR,ERROR


Update data in columns 'Total Number of Shares', 'IPO Number of Shares', and 'IPO Shares Percentage' with value = 'ERROR' to 0

In [206]:
df_stocks['Total Number of Shares'] = np.where(df_stocks['Total Number of Shares'] == 'ERROR',
                                               0, df_stocks['Total Number of Shares'])
df_stocks['IPO Number of Shares'] = np.where(df_stocks['IPO Number of Shares'] == 'ERROR',
                                               0, df_stocks['IPO Number of Shares'])
df_stocks['IPO Shares Percentage'] = np.where(df_stocks['IPO Shares Percentage'] == 'ERROR',
                                               0, df_stocks['IPO Shares Percentage'])

df_stocks['Total Number of Shares'] = pd.to_numeric(df_stocks["Total Number of Shares"])
df_stocks['IPO Number of Shares'] = pd.to_numeric(df_stocks["IPO Number of Shares"])
df_stocks['Owner Shares Amount'] = pd.to_numeric(df_stocks["Owner Shares Amount"])


##2) Fix 'IPO Price'

In [207]:
#data from IDNFinancial
df_stocks[df_stocks['IPO Price_x'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
10,11,CYBR,PT. ITSEC Asia Tbk,Teknologi,Jasa dan Konsultan Teknologi Informasi,175,08/08/2023,0,1008734800,5440839902,...,-,-,IT Services & Consulting,IT Services & Consulting,-,-,100,1008734800,6449574702,15.64
471,472,SKBM,PT. Sekar Bumi Tbk,Barang Konsumen Primer,Makanan Olahan,318,01/05/1993,0,7500000,31000000,...,Consumer Goods Industry,Food & Beverages,-,-,D. Consumer Non-Cyclicals,D2. Food & Beverage,1000,31000000,69500000,44.60
612,613,INPP,PT. Indonesian Paradise Property Tbk,Properti dan Perumahan,Pengelola dan Pengembang Real Estat,452,12/01/2004,0,0,1594000000,...,"Trade, Services & Investment","Restaurant, Hotel & Tourism",-,-,H. Properties & Real Estate,H1. Properties & Real Estate,100,240000000,1594000000,15.06
824,825,INRU,PT. Toba Pulp Lestari Tbk,Barang Baku,Perhutanan dan Kertas,995,06/18/1990,0,0,1325973625,...,Basic Industry & Chemicals,Pulp & Paper,-,-,B. Basic Materials,B1. Basic Materials,500,270000000,1325973625,20.36


In [208]:
#Data from SahamIdx
df_stocks[df_stocks['IPO Price_y'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage


In [209]:
df_stocks[(df_stocks['Stock Code'] == 'INPP')  | (df_stocks['Stock Code'] == 'INRU') | (df_stocks['Stock Code'] == 'SKBM') ]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
471,472,SKBM,PT. Sekar Bumi Tbk,Barang Konsumen Primer,Makanan Olahan,318,01/05/1993,0,7500000,31000000,...,Consumer Goods Industry,Food & Beverages,-,-,D. Consumer Non-Cyclicals,D2. Food & Beverage,1000,31000000,69500000,44.60
612,613,INPP,PT. Indonesian Paradise Property Tbk,Properti dan Perumahan,Pengelola dan Pengembang Real Estat,452,12/01/2004,0,0,1594000000,...,"Trade, Services & Investment","Restaurant, Hotel & Tourism",-,-,H. Properties & Real Estate,H1. Properties & Real Estate,100,240000000,1594000000,15.06
824,825,INRU,PT. Toba Pulp Lestari Tbk,Barang Baku,Perhutanan dan Kertas,995,06/18/1990,0,0,1325973625,...,Basic Industry & Chemicals,Pulp & Paper,-,-,B. Basic Materials,B1. Basic Materials,500,270000000,1325973625,20.36


after checking to:
- https://www.idx.co.id/Portals/0/StaticData/ListedCompanies/PerformanceSummary/INPP.pdf
- https://www.idx.co.id/Portals/0/StaticData/ListedCompanies/PerformanceSummary/SKBM.pdf
- https://www.idx.co.id/Portals/0/StaticData/ListedCompanies/PerformanceSummary/INRU.pdf

data from SahamIdx also wrong... so update manually


In [210]:
df_stocks['IPO Price_x'] = np.where(df_stocks['Stock Code'] == 'INRU', 9850, df_stocks['IPO Price_x'])
df_stocks['IPO Price_x'] = np.where(df_stocks['Stock Code'] == 'INPP', 105, df_stocks['IPO Price_x'])
df_stocks['IPO Price_x'] = np.where(df_stocks['Stock Code'] == 'SKBM', 5000, df_stocks['IPO Price_x'])

##3) Fix 'IPO New Shares Issued'

In [211]:
df_stocks[df_stocks['IPO New Shares Issued'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0
409,410,KOPI,PT. Mitra Energi Persada Tbk,Energi,Minyak dan Gas,382,05/04/2015,395,0,0,...,Infrastructure & Transportation,Energy,-,-,A. Energy,"A1. Oil, Gas & Coal",250,60000000,560000000,10.71
430,431,TALF,PT. Tunas Alfin Tbk,Barang Baku,Wadah dan Kemasan,340,01/17/2014,395,0,0,...,Basic Industry & Chemicals,Plastics & Packaging,-,-,B. Basic Materials,B1. Basic Materials,395,270000000,1353435000,19.95
598,599,ICON,PT. Island Concepts Indonesia Tbk,Perindustrian,Jasa Komersial,53,07/08/2005,113,0,125000000,...,"Trade, Services & Investment","Restaurant, Hotel & Tourism",-,-,C. Industrials,C2. Industrial Services,118,125000000,250000000,50.00
612,613,INPP,PT. Indonesian Paradise Property Tbk,Properti dan Perumahan,Pengelola dan Pengembang Real Estat,452,12/01/2004,105,0,1594000000,...,"Trade, Services & Investment","Restaurant, Hotel & Tourism",-,-,H. Properties & Real Estate,H1. Properties & Real Estate,100,240000000,1594000000,15.06
617,618,AGRO,PT. Bank Raya Indonesia Tbk,Keuangan,Bank,318,08/08/2003,100,0,1514043000,...,Finance,Bank,-,-,G. Financials,G1. Banks,100,768910000,3583913000,21.45
621,622,PSAB,PT. J Resources Asia Pasifik Tbk,Barang Baku,Logam dan Mineral,87,04/22/2003,250,0,30000000,...,Mining,Metal & Mineral Mining,-,-,B. Basic Materials,B1. Basic Materials,250,12000000,30000000,40.00
636,637,ABBA,PT. Mahaka Media Tbk,Barang Konsumen Bukan Primer,Media,83,04/03/2002,105,0,400000000,...,"Trade, Services & Investment","Advertising, Printing & Media",-,-,E. Consumer Cyclicals,E6. Media & Entertainment,105,240000000,640000000,37.50
681,682,SMDR,PT. Samudera Indonesia Tbk,Transportasi dan Logistik,Logistik dan Pengiriman,366,07/05/1999,500,0,163756000,...,Infrastructure & Transportation,Transportation,-,-,K. Transportation & Logistic,K2. Logistics & Deliveries,1000,13953334,177709334,7.85
689,690,SMMT,PT. Golden Eagle Energy Tbk,Energi,Batu Bara,1195,02/29/2000,500,0,80000000,...,Mining,Coal Mining,-,-,A. Energy,"A1. Oil, Gas & Coal",500,15000000,30000000,50.00


In [212]:
df_stocks[df_stocks['IPO Number of Shares'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
383,384,INCF,PT. Indo Komoditi Korpora Tbk,Barang Baku,Perhutanan dan Kertas,50,12/18/1989,8000,1200000,0,...,Basic Industry & Chemicals,Others Basic Industry & Chemicals,-,-,B. Basic Materials,B1. Basic Materials,ERROR,0,0,0
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0


if 'IPO New Shares Issued' from IDNFinancials = 0 then use data from IdxSaham, also recalculate 'Total Shares' and 'Owner Shares'

In [213]:
df_stocks['Total Shares Amount'] = np.where(df_stocks['IPO New Shares Issued'] == 0, df_stocks['Total Number of Shares'], df_stocks['Total Shares Amount'])
df_stocks['Owner Shares Amount'] = np.where(df_stocks['IPO New Shares Issued'] == 0, (df_stocks['Total Number of Shares'] - df_stocks['IPO Number of Shares']), df_stocks['Owner Shares Amount'])
df_stocks['IPO New Shares Issued'] = np.where(df_stocks['IPO New Shares Issued'] == 0, df_stocks['IPO Number of Shares'], df_stocks['IPO New Shares Issued'])

In [214]:
df_stocks[df_stocks['IPO New Shares Issued'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0


##4) Fix 'Total Shares Amount'

In [215]:
df_stocks[df_stocks['Total Shares Amount'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
383,384,INCF,PT. Indo Komoditi Korpora Tbk,Barang Baku,Perhutanan dan Kertas,50,12/18/1989,8000,1200000,0,...,Basic Industry & Chemicals,Others Basic Industry & Chemicals,-,-,B. Basic Materials,B1. Basic Materials,ERROR,0,0,0
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0
552,553,TPIA,PT. Chandra Asri Petrochemical Tbk,Barang Baku,Barang Kimia,2730,05/26/2008,2200,728401000,0,...,Basic Industry & Chemicals,Chemicals,-,-,B. Basic Materials,B1. Basic Materials,2575,257500000,728401000,35.35
589,590,RAJA,PT. Rukun Raharja Tbk,Energi,Minyak dan Gas,1010,04/19/2006,120,679513750,0,...,Infrastructure & Transportation,Energy,-,-,A. Energy,"A1. Oil, Gas & Coal",120,384859125,679513750,56.64
719,720,TLKM,PT. Telkom Indonesia (Persero) Tbk,Infrastruktur,Jasa Telekomunikasi,3800,11/14/1995,2050,933333000,0,...,Infrastructure & Transportation,Telecommunication,-,-,J. Infrastructures,J3. Telecommunication,2050,933333000,7466666000,12.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,MERK,PT. Merck Tbk,Kesehatan,Farmasi,4710,07/23/1981,1900,1680000,0,...,Consumer Goods Industry,Pharmaceuticals,-,-,F. Healthcare,F2. Pharmaceuticals & Health Care Research,1900,1680000,16800000,10.00
886,887,GDYR,PT. Goodyear Indonesia Tbk,Barang Konsumen Bukan Primer,Komponen Otomotif,0,12/22/1980,1250,6150000,0,...,Miscellaneous Industry,Automotive & Components,-,-,E. Consumer Cyclicals,E1. Automobiles & Components,1250,6150000,41000000,15.00
887,888,TFCO,PT. Tifico Fiber Indonesia Tbk,Barang Konsumen Bukan Primer,Pakaian dan Barang Mewah,0,02/26/1980,7250,1100000,0,...,Miscellaneous Industry,Textile & Garment,-,-,E. Consumer Cyclicals,E4. Apparel & Luxury Goods,7250,1100000,5500000,20.00
888,889,CNTX,PT. Century Textile Industry Tbk,Barang Konsumen Bukan Primer,Pakaian dan Barang Mewah,0,05/22/1979,5500,116000,0,...,Miscellaneous Industry,Textile & Garment,-,-,E. Consumer Cyclicals,E4. Apparel & Luxury Goods,5500,116000,700000,16.57


In [216]:
df_stocks[df_stocks['Total Number of Shares'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sector ID,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage
383,384,INCF,PT. Indo Komoditi Korpora Tbk,Barang Baku,Perhutanan dan Kertas,50,12/18/1989,8000,1200000,0,...,Basic Industry & Chemicals,Others Basic Industry & Chemicals,-,-,B. Basic Materials,B1. Basic Materials,ERROR,0,0,0
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Infrastructure & Transportation,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0


In [217]:
df_stocks['Recalculate Shares'] = np.where(df_stocks['Total Shares Amount'] == 0, 1, 0)
df_stocks['IPO New Shares Issued'] = np.where(df_stocks['Recalculate Shares'] == 1, df_stocks['IPO Number of Shares'], df_stocks['IPO New Shares Issued'])
df_stocks['Total Shares Amount'] = np.where(df_stocks['Recalculate Shares'] == 1, df_stocks['Total Number of Shares'], df_stocks['Total Shares Amount'])
df_stocks['Owner Shares Amount'] = np.where(df_stocks['Recalculate Shares'] == 1, (df_stocks['Total Number of Shares'] - df_stocks['IPO Number of Shares']), df_stocks['Owner Shares Amount'])

In [218]:
df_stocks[df_stocks['Total Shares Amount'] == 0]

,No,Stock Code,Company Name,Sector,Industry_x,Last Price,IPO Date,IPO Price_x,IPO New Shares Issued,Owner Shares Amount,...,Sub Sector ID,Industry_y,Sub Industry,Industry ID,Sub Industry ID,IPO Price_y,IPO Number of Shares,Total Number of Shares,IPO Shares Percentage,Recalculate Shares
383,384,INCF,PT. Indo Komoditi Korpora Tbk,Barang Baku,Perhutanan dan Kertas,50,12/18/1989,8000,0,0,...,Others Basic Industry & Chemicals,-,-,B. Basic Materials,B1. Basic Materials,ERROR,0,0,0,1
404,405,BUKK,PT. Bukaka Teknik Utama Tbk,Infrastruktur,Konstruksi Bangunan,1175,06/29/2015,590,0,0,...,Non Building Construction,-,-,J. Infrastructures,J2. Heavy Constructions & Civil Engineering,ERROR,0,0,0,1


##5) Recalculate 'IPO New Shares Issued Percentage'

In [219]:
df_stocks['IPO New Shares Issued Percentage'] = df_stocks['IPO New Shares Issued'] / df_stocks['Total Shares Amount']

##5) Calculate 'Owner Shares Percentage'

In [220]:
df_stocks['Owner Shares Percentage'] = df_stocks['Owner Shares Amount'] / df_stocks['Total Shares Amount']

#C) Merge Sector

In [221]:
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 889
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   No                                890 non-null    int64  
 1   Stock Code                        890 non-null    object 
 2   Company Name                      890 non-null    object 
 3   Sector                            890 non-null    object 
 4   Industry_x                        890 non-null    object 
 5   Last Price                        890 non-null    int64  
 6   IPO Date                          890 non-null    object 
 7   IPO Price_x                       890 non-null    int64  
 8   IPO New Shares Issued             890 non-null    int64  
 9   Owner Shares Amount               890 non-null    int64  
 10  Total Shares Amount               890 non-null    int64  
 11  IPO New Shares Issued Percentage  888 non-null    float64
 12  IPO Rais

In [222]:
df_StockSector_Final = pd.merge(df_stocks, df_StockSector, on='Stock Code')
df_StockSector_Final = df_StockSector_Final[['No', 'Stock Code', 'Company Name', 'Sector_y',  'Sub Sector',
                                 'Industry_x',  'Sub Industry', 'IPO Date', 'IPO Price_x', 'IPO New Shares Issued',
                                  'Owner Shares Amount', 'Total Shares Amount', 'IPO New Shares Issued Percentage',
                                  'Owner Shares Percentage', 'IPO Raised Fund', 'Board'
                                 ]].copy()
df_StockSector_Final = df_StockSector_Final.rename(columns={'Sector_y': 'Sector', 'Industry_x': 'Industry', 'IPO Price_x': 'IPO Price'})


In [223]:
df_StockSector_Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 889
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   No                                890 non-null    int64  
 1   Stock Code                        890 non-null    object 
 2   Company Name                      890 non-null    object 
 3   Sector                            890 non-null    object 
 4   Sub Sector                        890 non-null    object 
 5   Industry                          890 non-null    object 
 6   Sub Industry                      890 non-null    object 
 7   IPO Date                          890 non-null    object 
 8   IPO Price                         890 non-null    int64  
 9   IPO New Shares Issued             890 non-null    int64  
 10  Owner Shares Amount               890 non-null    int64  
 11  Total Shares Amount               890 non-null    int64  
 12  IPO New 

In [224]:
df_StockSector_Final.head()

,No,Stock Code,Company Name,Sector,Sub Sector,Industry,Sub Industry,IPO Date,IPO Price,IPO New Shares Issued,Owner Shares Amount,Total Shares Amount,IPO New Shares Issued Percentage,Owner Shares Percentage,IPO Raised Fund,Board
0,1,AEGS,PT. Anugerah Spareparts Sejahtera Tbk,Consumer Cyclical,Automobiles & Components,Komponen Otomotif,Auto Parts & Equipment,09/11/2023,100,400000000,606000000,1006000000,0.397614,0.602386,40000000000,Acceleration
1,2,BABY,PT. Multitrend Indo Tbk,Consumer Cyclical,Retailing,Ritel Khusus,Apparel & Textile Retail,09/07/2023,266,534000000,800000000,2668586400,0.200106,0.299784,142044000000,Main
2,3,RSCH,PT. Charlie Hospital Semarang Tbk,Healthcare,Healthcare Equipment & Providers,Penyedia Jasa Kesehatan,Healthcare Providers,08/28/2023,115,530000000,2120000000,2650000000,0.200000,0.800000,60950000000,Development
3,4,MSIE,PT. Multisarana Intan Eduka Tbk,Properties and Real Estates,Properties & Real Estate,Pengelola dan Pengembang Real Estat,Real Estate Development & Management,08/10/2023,100,360000000,1100000000,1460000000,0.246575,0.753425,36000000000,Acceleration
4,5,MUTU,PT. Mutuagung Lestari Tbk,Industrial,Industrial Services,Jasa Profesional,Research & Consulting Services,08/09/2023,108,942857200,2200000000,3142857200,0.300000,0.700000,101828577600,Development


#D) Merge Last Price (21 September 2023)

In [225]:
df_StockSector_Final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 889
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   No                                890 non-null    int64  
 1   Stock Code                        890 non-null    object 
 2   Company Name                      890 non-null    object 
 3   Sector                            890 non-null    object 
 4   Sub Sector                        890 non-null    object 
 5   Industry                          890 non-null    object 
 6   Sub Industry                      890 non-null    object 
 7   IPO Date                          890 non-null    object 
 8   IPO Price                         890 non-null    int64  
 9   IPO New Shares Issued             890 non-null    int64  
 10  Owner Shares Amount               890 non-null    int64  
 11  Total Shares Amount               890 non-null    int64  
 12  IPO New 

In [226]:
df_Stocks_final = pd.merge(df_StockSector_Final, df_LembarSahamLastPrice[['Stock Code', 'Stock Price']], on='Stock Code')

In [227]:
df_Stocks_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890 entries, 0 to 889
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   No                                890 non-null    int64  
 1   Stock Code                        890 non-null    object 
 2   Company Name                      890 non-null    object 
 3   Sector                            890 non-null    object 
 4   Sub Sector                        890 non-null    object 
 5   Industry                          890 non-null    object 
 6   Sub Industry                      890 non-null    object 
 7   IPO Date                          890 non-null    object 
 8   IPO Price                         890 non-null    int64  
 9   IPO New Shares Issued             890 non-null    int64  
 10  Owner Shares Amount               890 non-null    int64  
 11  Total Shares Amount               890 non-null    int64  
 12  IPO New 

#D) Underwriters

In [228]:
df_underwriters = df_IDNFinansialsIPOList[['Stock Code', 'Underwriters']].copy()

In [229]:
#DATA CLEANING
#special case, this 4 stocks underwriter didnt have delimiter so, manually add delimiter than proses like others
#CEKA	PT. Aspac Uppindo Sekuritas   PT. PDFCI Securities
#CMNP	PT. Indovest Securities PT. Pentasena Arthasentosa  PT. Usaha Bersama Sekuritas
#SMSM	PT. Asjaya Indosurya Securities  PT. Aspac Uppindo Sekuritas PT. Indosurya Securities PT. HG Asia Investment Holdings
#MITI	PT. Panin Sekuritas  PT. Pentasena Arthasentosa
df_underwriters.loc[df_underwriters['Stock Code'].str.contains('CEKA', case=True), 'Underwriters'] = 'PT. Aspac Uppindo Sekuritas, PT. PDFCI Securities'
df_underwriters.loc[df_underwriters['Stock Code'].str.contains('CMNP', case=True), 'Underwriters'] = 'PT. Indovest Securities, PT. Pentasena Arthasentosa,  PT. Usaha Bersama Sekuritas'
df_underwriters.loc[df_underwriters['Stock Code'].str.contains('SMSM', case=True), 'Underwriters'] = 'PT. Asjaya Indosurya Securities,  PT. Aspac Uppindo Sekuritas, PT. Indosurya Securities, PT. HG Asia Investment Holdings'
df_underwriters.loc[df_underwriters['Stock Code'].str.contains('MITI', case=True), 'Underwriters'] = 'PT. Panin Sekuritas,  PT. Pentasena Arthasentosa'

df_underwriters = df_underwriters.assign(Underwriters=df_underwriters['Underwriters'].str.split(','))\
    .explode('Underwriters')
df_underwriters['Underwriters'] = df_underwriters['Underwriters'].replace('\n', ' ', regex=True)
df_underwriters['Underwriters'] = df_underwriters['Underwriters']\
    .replace(['PT.', '(Affiliated)', '(affiliated)', '(Afilliated)','(Affilated)', 'Papan Pencatatan'],
             ' ', regex=True)
df_underwriters['Underwriters'] = df_underwriters['Underwriters']\
    .replace('[( )]',
             ' ', regex=True)

##Rename Underwriter

In [230]:
def renameunderwriter(stringtoreplace:str, stringreplaceby:str):
    df_underwriters.loc[df_underwriters['Underwriters'].str.contains(stringtoreplace, case=False), 'Underwriters'] = stringreplaceby

df_underwriters['Underwriters'] = df_underwriters['Underwriters'].str.strip()
renameunderwriter('ABN', 'ABN AMRO Asia Securities Indonesia')
renameunderwriter('Artha Securities', 'Artha Sekuritas Indonesia')
renameunderwriter('Aseam Indonesia', 'Asean Development Securities')
renameunderwriter('Asia kapitalindo Securities', 'Asia kapitalindo Securities')
renameunderwriter('BDNI Securities|Equity Development Securities', 'Equity Sekuritas Indonesia')
renameunderwriter('BNI', 'BNI Sekuritas')
renameunderwriter('BNP', 'BNP Sekuritas')
renameunderwriter('Danareksa|Danareka', 'BRI Danareksa Sekuritas')
renameunderwriter('Bahana', 'Bahana Sekuritas')
renameunderwriter('Bank Pembangunan Indonesia|Bapindo Bumi', 'Bapindo Bumi Sekuritas')
renameunderwriter('Baring', 'Baring Securities')
renameunderwriter('Bina Artha Sekuritas|Binaartha Parama', 'Binaartha Sekuritas')
renameunderwriter('Buana Capital', 'Buana Capital Sekuritas')
renameunderwriter('CIMB Niaga', 'CIMB Niaga Sekuritas')
renameunderwriter('CIMB Securities|CIMB Sekuritas|CIMB-GK Securities', 'PT CGS-CIMB Sekuritas Indonesia')
renameunderwriter('CLSA|CSLA|Credit Lyonnais', 'CLSA Sekuritas Indonesia')
renameunderwriter('Ciptadana', 'Ciptadana Sekuritas Asia')
renameunderwriter('Citicorp|Citigroup', 'Citigroup Sekuritas Indonesia')
renameunderwriter('Credit Suisse', 'Credit Suisse Sekuritas Indonesia')
renameunderwriter('DBS Securities Indonesia|DBS Vickers', 'DBS Vickers Sekuritas Indonesia')
renameunderwriter('Danasakti', 'Danasakti Sekuritas Indonesia')
renameunderwriter('Danatama Makmur', 'Danatama Makmur Sekuritas')
renameunderwriter('Dhanawibawa Arthacemerlang|Dhana Wibawa', 'Dhanawibawa Sekuritas Indonesia')
renameunderwriter('Dinamika Usahajaya', 'Dinamika Usaha Jaya')
renameunderwriter('Erdhika', 'Erdikha Elit Sekuritas')
renameunderwriter('Evergreen Capital', 'Evergreen Sekuritas Indonesia')
renameunderwriter('Ficonensia|Finconesia', 'Ficonesia')
renameunderwriter('HD Capital|Harumdana Sekuritas|Hortus Danavest', 'HD Capital')
renameunderwriter('Henan Putihrai', 'Henan Putihrai Sekuritas')
renameunderwriter('Indo Premier|Indopremier|Indopremir|Premier Securities', 'Indo Premier Sekuritas')
df_underwriters.loc[df_underwriters['Underwriters'].str.contains('IFI', case=True), 'Underwriters'] = 'Indonesian Finance and Investment Company (IFI)'
renameunderwriter('Indosurya Bersina|Indosurya Securities', 'Indosurya Bersinar Sekuritas')
df_underwriters.loc[df_underwriters['Underwriters'].isin(['Indovest', 'Indovest Securities', 'Indonesian Investments International']), 'Underwriters'] = 'Indonesian Investments International (Indovest)'
renameunderwriter('Inter Pacific|Inter- Pacific|Inter-Pacific', 'Inter Pacific Securities')
renameunderwriter('Investind Nusantara', 'Investindo Nusantara Sekuritas')
renameunderwriter('Jardine Fleming', 'J.P. Morgan Sekuritas Indonesia')
renameunderwriter('Jasa Utama Capital', 'Jasa Utama Capital Sekuritas')
renameunderwriter('KGI Sekuritas', 'KGI Sekuritas Indonesia')
renameunderwriter('Korea Investment & Sekuritas', 'Korea Investment and Sekuritas Indonesia')
renameunderwriter('Kresna', 'Kresna Graha Investama')
renameunderwriter('Lautandhana', 'Lautandhana Securindo')
renameunderwriter('MNC|Bhakti Securities', 'MNC Sekuritas')
renameunderwriter('Macquarie|Macquire', 'Macquarie Capital Securities Indonesia')
renameunderwriter('Makindo', 'Makindo Securities')
renameunderwriter('Merchant Investment Corporation|Merincop|Merincorp', 'Merchant Investment Corporation (Merincorp)')
renameunderwriter('Minna Padi ', 'Minna Padi Investama Sekuritas')
renameunderwriter('Mirae Asset Sekuritas', 'Mirae Asset Sekuritas Indonesia')
renameunderwriter('Morgan Grenfell', 'Deutsche Securities Indonesia')
renameunderwriter('Morgan Stanley', 'Morgan Stanley Sekuritas Indonesia')
renameunderwriter('Multicor', 'Multicorp')
renameunderwriter('Mutual Int', 'Mutual International Finance Corporation')
renameunderwriter('NH Korindo|NongHyup', 'NH Korindo Sekuritas Indonesia')
renameunderwriter('Nilai Inti Sekuritas', 'NISP Sekuritas')
renameunderwriter('Nusadana Capital Indonesia|OSK Nusadana|RHB', 'RHB OSK Securities Indonesia')
renameunderwriter('PDFCI', 'PDFCI Securities')
renameunderwriter('Pacific Capital|Pacific Sekuritas', 'Pacific Sekuritas Indonesia')
renameunderwriter('Panca Global', 'Panca Global Sekuritas')
renameunderwriter('Panin', 'Panin Sekuritas')
renameunderwriter('Reliance Sekuritas', 'Reliance Sekuritas Indonesia Tbk')
renameunderwriter('Rifan Financindo', 'Rifan Financindo Sekuritas')
renameunderwriter('Samuel Sekuritas', 'Samuel Sekuritas Indonesia')
renameunderwriter('Semesta Indovest', 'Semesta Indovest Sekuritas')
renameunderwriter('Sinar Mas', 'Sinarmas Sekuritas')
renameunderwriter('Sucor', 'Sucor Sekuritas')
renameunderwriter('Trimega', 'Trimegah Sekuritas Indonesia')
renameunderwriter('UBS', 'UBS Sekuritas Indonesia')
renameunderwriter('UOB', 'UOB Sekuritas Indonesia')
renameunderwriter('Valbury', 'KB Valbury Sekuritas')
renameunderwriter('Victoria', 'Victoria Sekuritas Indonesia')
renameunderwriter('W.I. Carr|WI Carr', 'W.I. Carr indonesia')
renameunderwriter('Wardley James Capel Indonesia', 'HSBC Sekuritas Indonesia')
renameunderwriter('Yuanta', 'Yuanta Sekuritas Indonesia')
renameunderwriter('Yulie', 'Yulie Sekuritas Indonesia')

#E) Save File to csv

In [231]:
from google.colab import drive
drive.mount('/content/drive')
df_Stocks_final.to_csv('/content/drive/My Drive/WORK ASSETS/Self Project/Indonesian Stock IPO Data Analysis/IPOStockList - Final.csv', index=False)
df_underwriters.to_csv('/content/drive/My Drive/WORK ASSETS/Self Project/Indonesian Stock IPO Data Analysis/IPOStockUnderwriters - Final.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
